# **UNIVERSIDAD DE INGENIERÍA Y TECNOLOGÍA (UTEC)**

## Maestría de Especialización en Ciencia de Datos & Inteligencia Artificial


## Curso: High-Performance Computing (HPC)

## Proyecto Final: Evolución Galáctica

## Profesor: José Antonio Fiestas Iquira

## Miembros:

*   Guedes del Pozo, Rodrigo Fernando
*   Salgado Paraguay, Julio César
*   Soriano Ricalde, Richard Clever

## Ciclo : IV


---


# **0. Breves Instrucciones de Uso**

Para la ejecución de la aplicación de análisis, es indispensable que el archivo de datos de rendimiento esté cargado en el entorno con el nombre exacto `Datos.csv`, el cual debe usar como separador el punto y coma (;). Los valores referidos deben ser los tiempos de ejecución en segundos. El archivo debe tener la siguiente estructura de columnas:

| proceso | 1K     | 2K      | 4K      | 8K       | 16K      | 32K       |
|---------|--------|---------|---------|----------|----------|-----------|
| 1       | 3.0215 | 13.3790 | 53.8510 | 142.8402 | 925.5528 | 4008.2889 |
| 2       | 1.6401 | 7.1824  | 16.3594 | 72.9454  | 516.8666 | 2506.5072 |
| 4       | 0.5244 | 3.5969  | 8.3563  | 38.2824  | 202.7420 | 878.5190  |
| 8       | 0.3468 | 2.2814  | 4.8136  | 20.4048  | 143.1880 | 488.7180  |
| 16      | 0.2333 | 1.4480  | 2.7478  | 10.8710  | 91.9410  | 397.5900  |
| 32      | 0.2998 | 0.9583  | 1.9794  | 7.1283   | 46.5200  | 129.3280  |



# **1. Análisis del Tiempo de Ejecución**

In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive, Checkbox, VBox, HBox
from IPython.display import display

# ⚠️ NOTA: Este código asume que el DataFrame 'datos_tiempo' fue cargado previamente
# y contiene la columna 'P' (procesos) y las columnas de tiempo ('1K', '2K', etc.).

# --- 1. Preparación de Datos: Carga y Transformación ---

RUTA_ARCHIVO = 'Datos.csv'
try:
    # Recrear datos_tiempo (por seguridad en la ejecución de Colab)
    datos_tiempo = pd.read_csv(RUTA_ARCHIVO, sep=';')
    datos_tiempo.rename(columns={'proceso': 'P'}, inplace=True)
except:
    raise FileNotFoundError("Error al cargar 'Datos.csv'.")

# Columnas de tamaño de problema
curvas_ordenadas = ['1K', '2K', '4K', '8K', '16K', '32K']

# Transformar de formato ancho a formato largo (melt)
df_tiempo_plot = datos_tiempo.melt(
    id_vars='P',
    value_vars=curvas_ordenadas,
    var_name='Tamaño_Problema (N)',
    value_name='Tej'
)

# --- 2. Definición de Colores y Estilos ---

COLORES_CURVAS = ['yellow', 'red', 'green', 'cyan', 'orange', 'purple']
mapeo_colores = dict(zip(curvas_ordenadas, COLORES_CURVAS))

# Estilo Oscuro
COLOR_FONDO_FIGURA = '#222222'
COLOR_FONDO_AXES = '#333333'
COLOR_TEXTO = 'white'
COLOR_REJILLA = '#555555'


## FUNCIÓN DE GRÁFICACIÓN INTERACTIVA DE TIEMPO
def plot_tiempo_interactivo(**kwargs):
    """
    Función que genera la gráfica de Tiempo de Ejecución basada en los controles de ipywidgets.
    """
    plt.figure(figsize=(11, 8), facecolor=COLOR_FONDO_FIGURA)
    ax = plt.gca()
    ax.set_facecolor(COLOR_FONDO_AXES)

    # ------------------ 1. Trazado de Curvas y Valores ------------------
    for tamaño in curvas_ordenadas:
        # Recuperar estado de los widgets
        mostrar_curva = kwargs.get(tamaño, False)
        mostrar_valores = kwargs.get(f'{tamaño}_values', False)

        subset = df_tiempo_plot[df_tiempo_plot['Tamaño_Problema (N)'] == tamaño]
        color_actual = mapeo_colores[tamaño]

        if mostrar_curva:
            # Trazar la línea
            ax.plot(
                subset['P'],
                subset['Tej'],
                linestyle='-',
                label=tamaño,
                color=color_actual,
                linewidth=3.5,
            )

            # Añadir Etiquetas de Datos (SOLO si se solicitó)
            if mostrar_valores:
                for x, y in zip(subset['P'], subset['Tej']):
                    ax.text(
                        x, y,
                        f'{y:.4f}',
                        ha='center',
                        va='bottom',
                        fontsize=9,
                        color='black',
                        # Recuadro blanco
                        bbox=dict(facecolor='white', alpha=1.0, edgecolor='black', boxstyle='round,pad=0.2')
                    )

    # ------------------ 2. Configuración de Estilo y Ejes ------------------

    # Ejes en escala Logarítmica (P y Tej)
    ax.set_yscale('log')
    ax.set_xscale('log')

    # Ajuste del rango 'Y'
    ax.set_ylim(0.1, 10000)

    # Eje X (Procesos)
    p_ticks = datos_tiempo['P'].unique()
    ax.set_xticks(p_ticks)
    ax.set_xticklabels(p_ticks, color=COLOR_TEXTO)
    ax.set_xlim(0.9, datos_tiempo['P'].max() * 1.05) # P=1 pegado al eje Y

    # Eje Y (Tiempo)
    ax.tick_params(axis='y', colors=COLOR_TEXTO)
    ax.get_yaxis().set_major_formatter(plt.FormatStrFormatter('%g'))

    # Títulos y etiquetas
    ax.set_title('Evolución Galáctica - Tiempo de Ejecución - MPI  (Khipu)',
                 fontsize=16, fontweight='bold', color=COLOR_TEXTO)
    ax.set_xlabel('P (# PROCESOS)', fontsize=12, color=COLOR_TEXTO)
    ax.set_ylabel('$T_{EJ}$ (SEC)', fontsize=12, color=COLOR_TEXTO)

    # Leyenda sin título
    if any(kwargs.get(t, False) for t in curvas_ordenadas):
        legend = ax.legend(loc='upper right', facecolor=COLOR_FONDO_AXES, title='')
        plt.setp(legend.get_texts(), color=COLOR_TEXTO)

    # Rejilla
    ax.grid(True, which="both", linestyle='-', linewidth=0.5, color=COLOR_REJILLA, alpha=0.9)
    ax.tick_params(axis='both', which='both', length=0)

    plt.tight_layout()
    plt.show()

## CREACIÓN DE WIDGETS (CONTROLES)
controles = {}
widgets_curva = []
widgets_valor = []

# Crear Checkboxes para cada curva y sus valores
for tamaño in curvas_ordenadas:
    # Checkbox para la curva
    chk_curva = Checkbox(
        value=True, # Activas por defecto
        description=f'Mostrar Curva {tamaño}',
        indent=False,
        layout={'width': '180px'}
    )
    controles[tamaño] = chk_curva
    widgets_curva.append(chk_curva)

    # Checkbox para los valores
    chk_valor = Checkbox(
        value=True, # Desactivados por defecto
        description=f'Mostrar Valores {tamaño}',
        indent=False,
        layout={'width': '180px'}
    )
    controles[f'{tamaño}_values'] = chk_valor
    widgets_valor.append(chk_valor)


# Organizar los widgets
columna_curvas = VBox(widgets_curva)
columna_valores = VBox(widgets_valor)

controles_container = HBox([columna_curvas, columna_valores])
controles_container.layout.border = '1px solid #555555'
controles_container.layout.padding = '10px'


# 🌟 Mostrar la interfaz interactiva 🌟
interactive_plot = interactive(plot_tiempo_interactivo, **controles)
display(controles_container, interactive_plot.children[-1])

Output()

# **2. Análisis del Speedup**

In [60]:
# --- 1. Preparación y Verificación de 'datos_tiempo' ---
# Asumo que 'datos_tiempo' ya existe y tiene la estructura correcta (como en el código anterior).

# Si estás ejecutando esto en una nueva celda de Colab, asegúrate de recrear o cargar
# 'datos_tiempo' primero. Usaremos el código de carga base para garantizar la ejecución.

RUTA_ARCHIVO = 'Datos.csv'
try:
    datos_tiempo = pd.read_csv(RUTA_ARCHIVO, sep=';')
    datos_tiempo.rename(columns={'proceso': 'P'}, inplace=True)
except:
    print("❌ Error al cargar 'Datos.csv'. Asegúrate de que el archivo esté cargado en Colab.")
    # Detener la ejecución si el DataFrame base no está disponible.
    raise

# Columnas de tiempo de ejecución
columnas_tej = ['1K', '2K', '4K', '8K', '16K', '32K']

# Verificar que las columnas existan
if not all(col in datos_tiempo.columns for col in columnas_tej):
    print(f"❌ Error: El DataFrame 'datos_tiempo' no contiene todas las columnas requeridas: {columnas_tej}")
    raise ValueError("Columnas no encontradas.")

# --- 2. Creación y Cálculo del DataFrame 'datos_speedup' ---

# 1. Crear una copia de 'datos_tiempo'
datos_speedup = datos_tiempo[['P'] + columnas_tej].copy()

# 2. Identificar la fila T1 (Tiempo de 1 Proceso)
# T1 es la primera fila (índice 0)
T1_row = datos_tiempo.iloc[0]

# 3. Calcular el Speedup para cada columna
for col in columnas_tej:
    # T1_value es el primer valor de la columna (Tiempo con P=1)
    T1_value = T1_row[col]

    # Speedup = T1 / T_P. La operación se aplica a toda la columna (Broadcasting).
    datos_speedup[col] = T1_value / datos_speedup[col]

    # Redondear a 4 decimales
    datos_speedup[col] = datos_speedup[col].round(4)

# --- 3. Agregar la Columna de Speedup Ideal ---

# El Speedup ideal S=O(p) es igual al número de procesos P
datos_speedup['S=O(p)'] = datos_speedup['P']

# --- Resultado Final ---

print("✅ DataFrame 'datos_speedup' generado y calculado correctamente.")
print("\nPrimeras filas del DataFrame 'datos_speedup':")
print(datos_speedup)

# Puedes usar el siguiente comando para ver la información general del nuevo DataFrame:
# datos_speedup.info()

✅ DataFrame 'datos_speedup' generado y calculado correctamente.

Primeras filas del DataFrame 'datos_speedup':
    P       1K       2K       4K       8K      16K      32K  S=O(p)
0   1   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000       1
1   2   1.8423   1.8627   3.2917   1.9582   1.7907   1.5992       2
2   4   5.7618   3.7196   6.4444   3.7312   4.5652   4.5626       4
3   8   8.7125   5.8644  11.1873   7.0003   6.4639   8.2016       8
4  16  12.9511   9.2396  19.5979  13.1396  10.0668  10.0815      16
5  32  10.0784  13.9612  27.2057  20.0385  19.8958  30.9932      32


In [64]:
# ⚠️ NOTA: Asegúrate de que los DataFrames 'datos_tiempo' y 'datos_speedup'
# estén definidos en una celda anterior antes de ejecutar este código.
# Para garantizar la ejecución, incluyo la recreación de 'datos_speedup' aquí:

RUTA_ARCHIVO = 'Datos.csv'
try:
    datos_tiempo = pd.read_csv(RUTA_ARCHIVO, sep=';')
    datos_tiempo.rename(columns={'proceso': 'P'}, inplace=True)
except:
    print("❌ Error al cargar 'Datos.csv'.")
    raise

columnas_tej = ['1K', '2K', '4K', '8K', '16K', '32K']
datos_speedup = datos_tiempo[['P'] + columnas_tej].copy()
T1_row = datos_tiempo.iloc[0]

for col in columnas_tej:
    T1_value = T1_row[col]
    datos_speedup[col] = (T1_value / datos_speedup[col]).round(4)

datos_speedup['S=O(p)'] = datos_speedup['P']

# --- Preparación para la Gráfica Interactiva ---

curvas_ordenadas = ['1K', '2K', '4K', '8K', '16K', '32K']
COLORES_CURVAS = ['yellow', 'red', 'green', 'cyan', 'orange', 'purple']
mapeo_colores = dict(zip(curvas_ordenadas, COLORES_CURVAS))
mapeo_colores['S=O(P) Ideal'] = 'white'

# Estilo Oscuro
COLOR_FONDO_FIGURA = '#222222'
COLOR_FONDO_AXES = '#333333'
COLOR_TEXTO = 'white'
COLOR_REJILLA = '#555555'

# Dataframes transformados para graficar
columnas_speedup = ['1K', '2K', '4K', '8K', '16K', '32K']
df_speedup_melted = datos_speedup.melt(
    id_vars='P',
    value_vars=columnas_speedup,
    var_name='Tamaño_Problema (N)',
    value_name='Speedup'
)
df_ideal = datos_speedup[['P', 'S=O(p)']].rename(columns={'S=O(p)': 'Speedup'})
df_ideal['Tamaño_Problema (N)'] = 'S=O(P) Ideal'
df_speedup_plot = pd.concat([df_speedup_melted, df_ideal], ignore_index=True)
orden_plot = ['S=O(P) Ideal'] + curvas_ordenadas


## FUNCIÓN DE GRÁFICACIÓN INTERACTIVA
def plot_speedup_interactivo(**kwargs):
    """
    Función que genera la gráfica de Speedup basada en los controles de ipywidgets.
    Los argumentos kwargs son generados por las casillas de verificación (Checkboxes).
    """
    plt.figure(figsize=(11, 8), facecolor=COLOR_FONDO_FIGURA)
    ax = plt.gca()
    ax.set_facecolor(COLOR_FONDO_AXES)

    # ------------------ 1. Trazado de Curvas y Valores ------------------
    for tamaño in orden_plot:
        # La clave en kwargs es el nombre del tamaño de problema (ej: '1K', '2K')
        # La clave para los valores de datos es '{tamaño}_values'
        mostrar_curva = kwargs.get(tamaño, False)
        mostrar_valores = kwargs.get(f'{tamaño}_values', False)
        is_ideal = (tamaño == 'S=O(P) Ideal')

        subset = df_speedup_plot[df_speedup_plot['Tamaño_Problema (N)'] == tamaño]
        color_actual = mapeo_colores[tamaño]

        if mostrar_curva:
            # Trazar la línea
            ax.plot(
                subset['P'],
                subset['Speedup'],
                linestyle='--' if is_ideal else '-',
                label=tamaño,
                color=color_actual,
                linewidth=3.5 if not is_ideal else 3.5,
            )

            # Añadir Etiquetas de Datos (SOLO si se solicitó Y no es la curva ideal)
            if mostrar_valores and not is_ideal:
                for x, y in zip(subset['P'], subset['Speedup']):
                    ax.text(
                        x, y,
                        f'{y:.4f}',
                        ha='center',
                        va='bottom',
                        fontsize=9,
                        color='black',
                        # Recuadro blanco
                        bbox=dict(facecolor='white', alpha=1.0, edgecolor='black', boxstyle='round,pad=0.2')
                    )

    # ------------------ 2. Configuración de Estilo ------------------

    # Ejes
    ax.set_yscale('log')
    ax.set_xscale('log')

    # Ajuste del rango 'Y'
    ax.set_ylim(1, 100)

    # Eje X (Procesos)
    p_ticks = datos_speedup['P'].unique()
    ax.set_xticks(p_ticks)
    ax.set_xticklabels(p_ticks, color=COLOR_TEXTO)
    ax.set_xlim(0.9, datos_speedup['P'].max() * 1.05)

    # Eje Y (Speedup)
    ax.tick_params(axis='y', colors=COLOR_TEXTO)
    ax.get_yaxis().set_major_formatter(plt.FormatStrFormatter('%g'))

    # Títulos y etiquetas
    ax.set_title('Evolución Galáctica - Speedup - MPI (Khipu)',
                 fontsize=16, fontweight='bold', color=COLOR_TEXTO)
    ax.set_xlabel('P (# PROCESOS)', fontsize=12, color=COLOR_TEXTO)
    ax.set_ylabel('S', fontsize=12, color=COLOR_TEXTO)

    # Leyenda sin título (solo si hay al menos una curva activa)
    if any(kwargs.get(t, False) for t in orden_plot):
        legend = ax.legend(loc='upper left', facecolor=COLOR_FONDO_AXES, title='')
        plt.setp(legend.get_texts(), color=COLOR_TEXTO)

    # Rejilla
    ax.grid(True, which="both", linestyle='-', linewidth=0.5, color=COLOR_REJILLA, alpha=0.9)
    ax.tick_params(axis='both', which='both', length=0)

    plt.tight_layout()
    plt.show()

## CREACIÓN DE WIDGETS (CONTROLES)
controles = {}
widgets_curva = []
widgets_valor = []

# Crear Checkboxes para cada curva y sus valores
for tamaño in orden_plot:
    # Checkbox para la curva
    chk_curva = Checkbox(
        value=True, # Activas por defecto
        description=f'Mostrar Curva {tamaño}',
        indent=False,
        layout={'width': '180px'}
    )
    controles[tamaño] = chk_curva
    widgets_curva.append(chk_curva)

    # Checkbox para los valores (Solo para curvas calculadas, no la ideal)
    if tamaño != 'S=O(P) Ideal':
        chk_valor = Checkbox(
            value=False, # Desactivados por defecto
            description=f'Mostrar Valores {tamaño}',
            indent=False,
            layout={'width': '180px'}
        )
        controles[f'{tamaño}_values'] = chk_valor
        widgets_valor.append(chk_valor)
    else:
        # Agregar un espacio en blanco para mantener la alineación de la curva ideal
        widgets_valor.append(widgets_curva[0].layout)


# Organizar los widgets en columnas
columna_curvas = VBox(widgets_curva)
columna_valores = VBox(widgets_valor)

# Contenedor principal para los controles
controles_container = HBox([columna_curvas, columna_valores])
controles_container.layout.border = '1px solid #555555'
controles_container.layout.padding = '10px'


# 🌟 Mostrar la interfaz interactiva 🌟
interactive_plot = interactive(plot_speedup_interactivo, **controles)
display(controles_container, interactive_plot.children[-1])

Output()

# **3. Análisis de la Eficiencia**

In [65]:
# --- 1. Recreación del DataFrame 'datos_speedup' (Necesario para la continuidad) ---
# Asumo que 'Datos.csv' está cargado.

RUTA_ARCHIVO = 'Datos.csv'
try:
    datos_tiempo = pd.read_csv(RUTA_ARCHIVO, sep=';')
    datos_tiempo.rename(columns={'proceso': 'P'}, inplace=True)
except:
    raise FileNotFoundError("Error al cargar 'Datos.csv'.")

columnas_tej = ['1K', '2K', '4K', '8K', '16K', '32K']
datos_speedup = datos_tiempo[['P'] + columnas_tej].copy()
T1_row = datos_tiempo.iloc[0]

for col in columnas_tej:
    T1_value = T1_row[col]
    datos_speedup[col] = (T1_value / datos_speedup[col]).round(4)
# La columna 'S=O(p)' no se necesita para el cálculo de eficiencia, pero la mantenemos por si acaso.
datos_speedup['S=O(p)'] = datos_speedup['P']


# --- 2. Creación y Cálculo del DataFrame 'datos_eficiencia' ---

# 1. Crear una copia de 'datos_speedup' (solo las columnas relevantes de Speedup y P)
datos_eficiencia = datos_speedup[['P'] + columnas_tej].copy()

# 2. Calcular la Eficiencia (Eficiencia = Speedup / P)
for col in columnas_tej:
    # Dividir cada columna de Speedup (datos_eficiencia[col])
    # por la columna de Procesos (datos_eficiencia['P'])
    datos_eficiencia[col] = datos_eficiencia[col] / datos_eficiencia['P']

    # Redondear a 4 decimales
    datos_eficiencia[col] = datos_eficiencia[col].round(4)

# --- 3. Agregar la Columna de Eficiencia Ideal ---

# El valor de la Eficiencia Ideal es 1 (E=1).
# Rellenar la última columna con el valor 'uno' (1.0) y nombre 'E=O(1)'
datos_eficiencia['E=O(1)'] = 1.0

# --- Resultado Final ---

print("✅ DataFrame 'datos_eficiencia' generado y calculado correctamente.")
print("\nPrimeras filas del DataFrame 'datos_eficiencia':")
print(datos_eficiencia.head(10))

✅ DataFrame 'datos_eficiencia' generado y calculado correctamente.

Primeras filas del DataFrame 'datos_eficiencia':
    P      1K      2K      4K      8K     16K     32K  E=O(1)
0   1  1.0000  1.0000  1.0000  1.0000  1.0000  1.0000     1.0
1   2  0.9212  0.9314  1.6458  0.9791  0.8954  0.7996     1.0
2   4  1.4404  0.9299  1.6111  0.9328  1.1413  1.1406     1.0
3   8  1.0891  0.7330  1.3984  0.8750  0.8080  1.0252     1.0
4  16  0.8094  0.5775  1.2249  0.8212  0.6292  0.6301     1.0
5  32  0.3150  0.4363  0.8502  0.6262  0.6217  0.9685     1.0


In [66]:
# @title
# ⚠️ NOTA: Este código asume que el DataFrame 'datos_eficiencia' está definido.

# --- 1. Preparación de Datos: Definición de Estilos ---

curvas_ordenadas = ['1K', '2K', '4K', '8K', '16K', '32K']
COLORES_CURVAS = ['yellow', 'red', 'green', 'cyan', 'orange', 'purple']
mapeo_colores = dict(zip(curvas_ordenadas, COLORES_CURVAS))
mapeo_colores['E=O(1)'] = 'white'

# Estilo Oscuro
COLOR_FONDO_FIGURA = '#222222'
COLOR_FONDO_AXES = '#333333'
COLOR_TEXTO = 'white'
COLOR_REJILLA = '#555555'

# Dataframes transformados a formato largo (melt)
columnas_eficiencia = ['1K', '2K', '4K', '8K', '16K', '32K']
df_eficiencia_melted = datos_eficiencia.melt(
    id_vars='P',
    value_vars=columnas_eficiencia,
    var_name='Tamaño_Problema (N)',
    value_name='Eficiencia'
)

# Agregar la línea de Eficiencia Ideal (E=O(1))
df_ideal = datos_eficiencia[['P', 'E=O(1)']].rename(columns={'E=O(1)': 'Eficiencia'})
df_ideal['Tamaño_Problema (N)'] = 'E=O(1)'

df_eficiencia_plot = pd.concat([df_eficiencia_melted, df_ideal], ignore_index=True)
orden_plot = ['E=O(1)'] + curvas_ordenadas


## FUNCIÓN DE GRÁFICACIÓN INTERACTIVA DE EFICIENCIA
def plot_eficiencia_interactivo(**kwargs):
    """
    Función que genera la gráfica de Eficiencia basada en los controles de ipywidgets.
    """
    plt.figure(figsize=(11, 8), facecolor=COLOR_FONDO_FIGURA)
    ax = plt.gca()
    ax.set_facecolor(COLOR_FONDO_AXES)

    # ------------------ 1. Trazado de Curvas y Valores ------------------
    for tamaño in orden_plot:
        mostrar_curva = kwargs.get(tamaño, False)
        mostrar_valores = kwargs.get(f'{tamaño}_values', False)
        is_ideal = (tamaño == 'E=O(1)')

        subset = df_eficiencia_plot[df_eficiencia_plot['Tamaño_Problema (N)'] == tamaño]
        color_actual = mapeo_colores[tamaño]

        if mostrar_curva:
            # Trazar la línea
            ax.plot(
                subset['P'],
                subset['Eficiencia'],
                linestyle='--' if is_ideal else '-',
                label=tamaño,
                color=color_actual,
                linewidth=3.5 if not is_ideal else 3.5,
            )

            # Añadir Etiquetas de Datos (SOLO si se solicitó Y no es la curva ideal)
            if mostrar_valores and not is_ideal:
                for x, y in zip(subset['P'], subset['Eficiencia']):
                    ax.text(
                        x, y,
                        f'{y:.4f}',
                        ha='center',
                        va='bottom',
                        fontsize=9,
                        color='black',
                        # Recuadro blanco
                        bbox=dict(facecolor='white', alpha=1.0, edgecolor='black', boxstyle='round,pad=0.2')
                    )

    # ------------------ 2. Configuración de Estilo y Ejes ------------------

    # Ejes
    ax.set_yscale('log')
    ax.set_xscale('log')

    # Ajuste del rango 'Y'
    ax.set_ylim(0.10, 10)
    # Configurar los ticks para que muestren las potencias de 10 visibles en el rango
    ax.set_yticks([0.1, 1.0, 10.0])

    # Eje X (Procesos)
    p_ticks = datos_eficiencia['P'].unique()
    ax.set_xticks(p_ticks)
    ax.set_xticklabels(p_ticks, color=COLOR_TEXTO)
    ax.set_xlim(0.9, datos_eficiencia['P'].max() * 1.05)

    # Eje Y (Eficiencia)
    ax.tick_params(axis='y', colors=COLOR_TEXTO)
    ax.get_yaxis().set_major_formatter(plt.FormatStrFormatter('%g'))

    # Títulos y etiquetas
    ax.set_title('Evolución Galáctica - Eficiencia - MPI (Khipu)',
                 fontsize=16, fontweight='bold', color=COLOR_TEXTO)
    ax.set_xlabel('P (# PROCESOS)', fontsize=12, color=COLOR_TEXTO)
    ax.set_ylabel('E', fontsize=12, color=COLOR_TEXTO)

    # Leyenda sin título
    if any(kwargs.get(t, False) for t in orden_plot):
        legend = ax.legend(loc='upper left', facecolor=COLOR_FONDO_AXES, title='')
        plt.setp(legend.get_texts(), color=COLOR_TEXTO)

    # Rejilla
    ax.grid(True, which="both", linestyle='-', linewidth=0.5, color=COLOR_REJILLA, alpha=0.9)
    ax.tick_params(axis='both', which='both', length=0)

    plt.tight_layout()
    plt.show()

## CREACIÓN DE WIDGETS (CONTROLES)
controles = {}
widgets_curva = []
widgets_valor = []

# Crear Checkboxes para cada curva y sus valores
for tamaño in orden_plot:
    chk_curva = Checkbox(
        value=True,
        description=f'Mostrar Curva {tamaño}',
        indent=False,
        layout={'width': '180px'}
    )
    controles[tamaño] = chk_curva
    widgets_curva.append(chk_curva)

    if tamaño != 'E=O(1)':
        chk_valor = Checkbox(
            value=False,
            description=f'Mostrar Valores {tamaño}',
            indent=False,
            layout={'width': '180px'}
        )
        controles[f'{tamaño}_values'] = chk_valor
        widgets_valor.append(chk_valor)
    else:
        widgets_valor.append(widgets_curva[0].layout)


# Organizar los widgets
columna_curvas = VBox(widgets_curva)
columna_valores = VBox(widgets_valor)

controles_container = HBox([columna_curvas, columna_valores])
controles_container.layout.border = '1px solid #555555'
controles_container.layout.padding = '10px'


# 🌟 Mostrar la interfaz interactiva 🌟
interactive_plot = interactive(plot_eficiencia_interactivo, **controles)
display(controles_container, interactive_plot.children[-1])

Output()

# **4. Análisis de la Escalabilidad**



In [71]:
# ⚠️ NOTA: Asumimos que 'datos_eficiencia' ya fue calculado y está disponible.

# --- 1. CONFIGURACIÓN ---

# Define el umbral de "buena" escalabilidad.
# Si la eficiencia final es mayor a este valor, se considera buena.
UMBRAL_EFICIENCIA = 0.75
columnas_eficiencia = ['1K', '2K', '4K', '8K', '16K', '32K']

# El número máximo de procesos utilizados (la última fila)
P_max = datos_eficiencia['P'].max()

print(f"--- Diagnóstico de Escalabilidad Fuerte (P máximo = {P_max}) ---")
print(f"Umbral de Eficiencia para 'Buena Escalabilidad': > {UMBRAL_EFICIENCIA}\n")


# --- 2. ANÁLISIS DE ESCALABILIDAD FUERTE ---

resultados_fuerte = []

for N in columnas_eficiencia:
    # Obtener la Eficiencia en el máximo número de procesos (P_max)
    # Buscamos el valor en la última fila del dataframe (índice -1)
    # Excluimos la columna 'E=O(1)' si existe
    if N in datos_eficiencia.columns:
        eficiencia_final = datos_eficiencia[N].iloc[-1]

        if eficiencia_final >= UMBRAL_EFICIENCIA:
            diagnostico = f"Buena Escalabilidad. (E = {eficiencia_final:.4f} ≈ 1)"
        elif eficiencia_final >= 0.50:
            diagnostico = f"Aceptable. (E = {eficiencia_final:.4f})"
        else:
            diagnostico = f"Pobre. Caída significativa de eficiencia. (E = {eficiencia_final:.4f})"

        # Añadir la conclusión al resultado
        resultados_fuerte.append({
            'Tamaño del Problema (N)': N,
            f'Eficiencia en P={P_max}': f"{eficiencia_final:.4f}",
            'Diagnóstico': diagnostico
        })

# --- 3. PRESENTACIÓN DE RESULTADOS ---

df_diagnostico = pd.DataFrame(resultados_fuerte)
df_diagnostico.set_index('Tamaño del Problema (N)', inplace=True)

print("📋 Conclusiones de Escalabilidad Fuerte por Tamaño de Problema Fijo:")
print(df_diagnostico)

# --- 4. DIAGNÓSTICO SOBRE ESCALABILIDAD DÉBIL ---

print("\n--- Diagnóstico sobre Escalabilidad Débil ---")
print("❌ **Limitación de Datos:** Los datos actuales (P vs N fijo) están estructurados para el análisis de Escalabilidad Fuerte.")
print("Para determinar la Escalabilidad Débil (N/P = constante), necesitarías filas de datos donde:")
print("* Paso 1: P=1, N=1K")
print("* Paso 2: P=2, N=2K (N/P se mantiene en 1K)")
print("* Paso 3: P=4, N=4K (N/P se mantiene en 1K)")
print("\nDado que la estructura actual es (P=1 a 32, N=1K), (P=1 a 32, N=2K), etc., no es posible evaluarla formalmente. Sin embargo, si el tiempo de ejecución **mínimo** para cada tamaño de problema (1K, 2K, 4K...) no aumenta significativamente al movernos de un tamaño fijo a otro, podría haber una indicación de buena escalabilidad débil.")

--- Diagnóstico de Escalabilidad Fuerte (P máximo = 32) ---
Umbral de Eficiencia para 'Buena Escalabilidad': > 0.75

📋 Conclusiones de Escalabilidad Fuerte por Tamaño de Problema Fijo:
                        Eficiencia en P=32  \
Tamaño del Problema (N)                      
1K                                  0.3150   
2K                                  0.4363   
4K                                  0.8502   
8K                                  0.6262   
16K                                 0.6217   
32K                                 0.9685   

                                                               Diagnóstico  
Tamaño del Problema (N)                                                     
1K                       Pobre. Caída significativa de eficiencia. (E =...  
2K                       Pobre. Caída significativa de eficiencia. (E =...  
4K                                   Buena Escalabilidad. (E = 0.8502 ≈ 1)  
8K                                                 Aceptable.